In [1]:
import kfp
import kfp.dsl as dsl
import kfp.onprem as onprem
import kfp.components as comp

# for tensorboard
from kubernetes.client.models import V1EnvVar
    
@dsl.pipeline(
    name='tensorboardPipeline',
    description='tensorboard'
)

def tensorboard_pipeline(tb_log_dir):

    vop = dsl.VolumeOp(
        name="volume_creation",
        resource_name="vol-a",
        storage_class="nfs-client", # here
        modes=dsl.VOLUME_MODE_RWO,
        size="1Gi"
    )
    
    cop1 = dsl.ContainerOp(
        name='tensorboard',
        image='kubeflow-registry.default.svc.cluster.local:30000/tensorboard-job:2BCB9C7F',  # here
        command=['python', '/app/02-train-fairing.py']
    )
    
    cop1.after(vop) # pipeline code

if __name__ == '__main__':
    arguments = {'tb_log_dir': 's3://tensorboard/mnist01/'}
    my_run = kfp.Client().create_run_from_pipeline_func(tensorboard_pipeline, arguments=arguments, experiment_name='tensorboard Experiment')